In [1]:
import argparse

import torch
from torchvision import utils
import torchvision
import torchvision.models as tvm
from torchvision import transforms
from PIL import Image
import numpy as np
import sys
import torch
from torch.utils.data import Dataset
import os
from PIL import Image
from torchvision import transforms
import math
from torch import nn
import torchvision.models as models
from tqdm import tqdm
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import json

In [2]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

In [148]:
def image_class_probabilities():
    predict_dict = []
    all_predictions = []
    dataset = ImageFolder('/common/home/ssa162/CS535/stylegan2-interface/output_if', transform=transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    device = torch.device('cuda:0')   
    resnet = Classifier()
    resnet = resnet.to(device)
    resnet.load_state_dict(torch.load('./model.pth',map_location='cuda:0')["state_dict"])
    resnet.eval()
    i=0
    with torch.no_grad():
        for image, _ in tqdm(dataloader):
            file_name = str(i)
            prediction = resnet(image.to(device))
            predicted_class = torch.sigmoid(prediction)
            predict_dict.extend(predicted_class.tolist())
            all_predictions.extend(predicted_class.tolist())
            if (i+1)%10 == 0:
                with open(os.path.join(os.getcwd(),"predictions",file_name+'.txt'), 'w+') as outfile:
                    json.dump(predict_dict, outfile)
                predict_dict = []
            i+=1
    with open(os.path.join(os.getcwd(),"predictions",'all.txt'), 'w+') as outfile:
        json.dump(all_predictions, outfile)
    return all_predictions

In [149]:
a=image_class_probabilities()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [03:26<00:00,  4.54it/s]


In [1]:
def calculate_interface_metric():
    predict_dict = []
    all_predictions = []
    dataset = ImageFolder('/common/home/ssa162/CS535/stylegan2-pytorch/MockClass/', transform=transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    device = torch.device('cuda:0')   
    resnet = Classifier()
    resnet = resnet.to(device)
    resnet.load_state_dict(torch.load('./model.pth',map_location='cuda:0')["state_dict"])
    resnet.eval()
    i=0
    with torch.no_grad():
        for image, _ in tqdm(dataloader):
            file_name = str(i)
            prediction = resnet(image.to(device))
            predicted_class = torch.sigmoid(prediction)
            predict_dict.extend(predicted_class.tolist())
            all_predictions.extend(predicted_class.tolist())
            if (i+1)%10 == 0:
                with open(os.path.join(os.getcwd(),"predictions",file_name+'.txt'), 'w+') as outfile:
                    json.dump(predict_dict, outfile)
                predict_dict = []
            i+=1
    with open(os.path.join(os.getcwd(),"predictions",'all.txt'), 'w+') as outfile:
        json.dump(all_predictions, outfile)
    return all_predictions

In [5]:

class PizzaData(Dataset):
    def __init__(self, base_dir='/common/home/ms3185/Sefa/Dataset', transforms=None, is_train=True):
        self.base_dir = base_dir
        label_file = os.path.join(base_dir, 'imageLabels.txt')
        with open(label_file) as f:
            self.labels = [x.strip().lower() for x in f.read().split('\n') if x]
        self.images_path = os.path.join(base_dir, 'images')
        self.is_train = is_train
        self.transforms = transforms
        self.thresh = int(len(self.labels) * 0.8)
        self.indices = list(range(self.thresh)) if self.is_train else list(range(self.thresh, len(self.labels)))
        
    def __len__(self):
        return len(self.indices)
    
    def create_tensor(self, label):
        return torch.tensor([float(x) for x in label.split()])
    
    def __getitem__(self, idx):
        idx = self.indices[idx]
        label = self.labels[idx]
        image_name = os.path.join(self.images_path, str(idx + 1).zfill(5) + '.jpg')
        image = Image.open(image_name).convert('RGB')
        return self.transforms(image), self.create_tensor(label)


# In[ ]:


class Classifier(nn.Module):
    def __init__(self, classes=10):
        super().__init__()
        self.main = models.resnet18(pretrained=True)
        num_feat = self.main.fc.in_features
        self.main.fc = nn.Linear(num_feat, classes)

    def forward(self, img):
        return self.main(img)

## Accuracy for images in the latent code (without movement)

In [14]:
predict_dict = []
all_predictions = []
dataset = ImageFolder('/common/home/ssa162/CS535/stylegan2-interface/Output_Negative', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
device = torch.device('cuda:0')   
resnet = Classifier()
resnet = resnet.to(device)
resnet.load_state_dict(torch.load('./model.pth',map_location='cuda:0')["state_dict"])
resnet.eval()
i=0
total_prob = {}
count_prob = {}
for i in range(10):
    total_prob[str(i)] = 0
    count_prob[str(i)] =0
    
with torch.no_grad():
    for image, classes in tqdm(dataloader):
        prediction = resnet(image.to(device))
        predicted_class = torch.sigmoid(prediction)
        predicted_class = predicted_class.tolist()
        for idx,class_val in enumerate(classes):
            clas_v = int(class_val)
            total_prob[str(clas_v)] +=predicted_class[idx][int(clas_v)]
            count_prob[str(clas_v)] +=1



100%|███████████████████████████████████████████████████████████████████████████████████| 79/79 [00:26<00:00,  3.02it/s]


In [13]:
for i in range(10):
    print(f"Accuracy of class {i}: {round(total_prob[str(i)]/count_prob[str(i)],2)}")
            

Accuracy of class 0: 0.34
Accuracy of class 1: 0.3
Accuracy of class 2: 0.31
Accuracy of class 3: 0.31
Accuracy of class 4: 0.2
Accuracy of class 5: 0.38
Accuracy of class 6: 0.47
Accuracy of class 7: 0.42
Accuracy of class 8: 0.17
Accuracy of class 9: 0.29


## Accuracy for images while moving in positive direction

In [40]:
predict_dict = []
all_predictions = []
dataset = ImageFolder('/common/home/ssa162/CS535/stylegan2-interface/output_if', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
device = torch.device('cuda:0')   
resnet = Classifier()
resnet = resnet.to(device)
resnet.load_state_dict(torch.load('./model.pth',map_location='cuda:0')["state_dict"])
resnet.eval()
i=0
total_prob = {}
count_prob = {}
for i in range(10):
    total_prob[str(i)] = 0
    count_prob[str(i)] =0
    
with torch.no_grad():
    for image, classes in tqdm(dataloader):
        prediction = resnet(image.to(device))
        predicted_class = torch.sigmoid(prediction)
        predicted_class = predicted_class.tolist()
        for idx,class_val in enumerate(classes):
            clas_v = int(class_val)
            total_prob[str(clas_v)] +=predicted_class[idx][int(clas_v)]
            count_prob[str(clas_v)] +=1



100%|███████████████████████████████████████████████████████████████████████████████████| 79/79 [00:23<00:00,  3.40it/s]


In [42]:
for i in range(10):
    print(f"Accuracy of class {i}: {round(total_prob[str(i)]/count_prob[str(i)],2)}")
            

Accuracy of class 0: 0.7
Accuracy of class 1: 0.58
Accuracy of class 2: 0.7
Accuracy of class 3: 0.68
Accuracy of class 4: 0.58
Accuracy of class 5: 0.87
Accuracy of class 6: 0.75
Accuracy of class 7: 0.71
Accuracy of class 8: 0.71
Accuracy of class 9: 0.82
